### Step 1. Open data file

In [1]:
import pandas as pd
import math

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer

try:
    df = pd.read_csv('../datasets/credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

display(df.info())
display(df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10375,1,-2041.666104,46,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,22963.430,purchase of a car
2107,1,-4000.099155,28,bachelor's degree,0,civil partnership,1,F,business,0,34348.526,education
14657,0,-2737.801185,38,bachelor's degree,0,married,0,F,employee,0,24902.419,cars
4518,0,-4448.175193,51,primary education,3,married,0,F,employee,0,13732.933,purchase of the house for my family
242,0,NaN,58,secondary education,1,married,0,F,civil servant,0,NaN,purchase of my own house


21525 entries, 12 columns of personal data of varying data types

### Step 2. Data preprocessing

#### Processing missing values

In [2]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are 2174 missing values in `day_employed` and `total_income` so I will drop those rows because they are not a significant part of the overall data.

In [3]:
df.dropna(inplace=True)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

The missing data values have been removed.

### Data type replacement

Based on the data description, `days_employed` and `total_income` are floats that should be integers.

In [4]:
try:
    df['days_employed'] = df['days_employed'].astype(int)
    df['total_income'] = df['total_income'].astype(int)
    df.info()
except:
    print('Error converting type')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          19351 non-null  int64 
 1   days_employed     19351 non-null  int32 
 2   dob_years         19351 non-null  int64 
 3   education         19351 non-null  object
 4   education_id      19351 non-null  int64 
 5   family_status     19351 non-null  object
 6   family_status_id  19351 non-null  int64 
 7   gender            19351 non-null  object
 8   income_type       19351 non-null  object
 9   debt              19351 non-null  int64 
 10  total_income      19351 non-null  int32 
 11  purpose           19351 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [5]:
# remove rows with 0 value in dob_years
df = df[df['dob_years'] != 0]

# columns (ex. children, days_employed) have negative values that need to be fixed
df['children'] = abs(df['children'])
df['days_employed'] = abs(df['days_employed'])

Now integer values properly represent the data. The issue of `dob_years` values being zero and `days_employed` & `children` having negative values should be brought to attention of those who provided the data.

In [6]:
# put string values in lowercase for consistency
text_columns = ['education','family_status','gender','income_type','purpose']

for col in text_columns:
    df[col] = df[col].str.lower()

df[text_columns].head(3)

,education,family_status,gender,income_type,purpose
0,bachelor's degree,married,f,employee,purchase of the house
1,secondary education,married,f,employee,car purchase
2,secondary education,married,m,employee,purchase of the house


### Processing duplicates

In [7]:
df.drop_duplicates(inplace=True)

We no longer have duplicate entries.

### Categorizing Data

We will categorize variables in order to answer later analysis questions.

In [8]:
# apply 1 if married or in civil partnership
def married (row):
    if (row['family_status_id'] == 0 or row['family_status_id'] == 1):
        return 1
    else:
        return 0
df['married'] = df.apply(married,axis=1)

# apply 0 if no children, 1 if have children
def have_children_col (row):
    if (row['children'] > 0):
        return 1
    else:
        return 0
df['have_children'] = df.apply(have_children_col,axis=1)

# apply income level by quartile
income_max = df['total_income'].max()
quarter = math.floor(income_max/4)
def income_level_col (row):
    if row['total_income'] <= quarter:
        return 'low'
    elif row['total_income']>quarter and row['total_income']<=(quarter*2):
        return 'mid'
    elif row['total_income']>(quarter*2) and row['total_income']<=(quarter*3):
        return 'high'
    else:
        return 'very high'
df['income_level'] = df.apply(income_level_col,axis=1)

# stem and lemmatize purpose strings to 4 categories:
# 0 for car-related purchases, 1 for education , 2 for wedding
# 3 for house/real-estate, 4 for other
wordnet_lemma = WordNetLemmatizer()
english_stemmer = SnowballStemmer('english')

def purpose_identify(row):
    words = nltk.word_tokenize(row.purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    for lemma in lemmas:
        stem = english_stemmer.stem(lemma)
        if stem == "car":
            return 0
        elif stem == 'educ' or stem == 'univers':
            return 1
        elif stem == 'wed' or stem == 'ceremoni':
            return 2
        elif stem == 'estat' or stem == 'properti' or stem == 'construct' or stem == 'hous':
            return 3
    return 4

df['purpose_number'] = df.apply(purpose_identify,axis=1)

df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,married,have_children,income_level,purpose_number
8761,2,609,28,bachelor's degree,0,married,0,m,business,0,36201,buy commercial real estate,1,1,low,3
6,0,2879,43,bachelor's degree,0,married,0,f,business,0,38484,housing transactions,1,0,low,3
14881,0,4656,58,secondary education,1,married,0,f,employee,0,23986,transactions with my real estate,1,0,low,3
880,0,758,48,some college,2,married,0,f,business,0,50184,buying a second-hand car,1,0,low,0
13482,1,2988,28,bachelor's degree,0,married,0,m,employee,0,33582,buying a second-hand car,1,1,low,0


- `children` and `married` was simplified to binary categorization
- `income_type` was categorized by quartiles to show an appropriate amount of division between income levels
- `purpose` was first lemmatized to extract meaningful words, then lemmatized to encode common purposes

### Step 3. Analysis questions

#### Is there a relation between having kids and repaying a loan on time?

In [23]:
pivot_table = df.groupby(['have_children']).agg({'debt':'mean'})
display(pivot_table)

,debt
have_children,
0,0.074868
1,0.093178


In [10]:
overall_total = df.count()[0]
overall_default = df.loc[df['debt'] == 1].count()[0]
overall_rate = overall_default / overall_total * 100
print('The overall default rate is: {:.2f}%'.format(overall_rate))
print('The default rate for people without children is: {:.2f}%'.format(pivot_table['debt'][0]*100))
print('The default rate for people with children is: {:.2f}%'.format(pivot_table['debt'][1]*100))

The overall default rate is: 8.12%
The default rate for people without children is: 7.49%
The default rate for people with children is: 9.32%


Those with children are slightly more likely to default than those without children. This is because children are a financial burden.

#### Is there a relation between marital status and repaying a loan on time?

In [25]:
pivot_table = df.groupby(['married']).agg({'debt':'mean'})
display(pivot_table)

,debt
married,
0,0.086389
1,0.079582


In [12]:
print('The overall default rate is: {:.2f}%'.format(overall_rate))
print('The default rate for unmarried people is: {:.2f}%'.format(pivot_table['debt'][0]*100))
print('The default rate for married people is: {:.2f}%'.format(pivot_table['debt'][1]*100))

The overall default rate is: 8.12%
The default rate for unmarried people is: 8.64%
The default rate for married people is: 7.96%


Those who are married are slightly less likely to default. This is because a couple can pool their incomes together.

#### Is there a relation between income level and repaying a loan on time?

In [27]:
pivot_table = df.groupby(['income_level']).agg({'debt':'mean'})
display(pivot_table)
display(df['income_level'].value_counts())

,debt
income_level,
high,0.000000
low,0.081311
mid,0.057377
very high,0.200000


low          19124
mid            122
high             9
very high        5
Name: income_level, dtype: int64

In [28]:
incomes = ['low','mid','high','very high']

print('The overall default rate is: {:.2f}%'.format(overall_rate))

for i in range(len(incomes)):
    print('The default rate for {:} income is: {:.2f}%'.format(incomes[i], pivot_table['debt'][i]*100))

The overall default rate is: 8.12%
The default rate for low income is: 0.00%
The default rate for mid income is: 8.13%
The default rate for high income is: 5.74%
The default rate for very high income is: 20.00%


The default rate is lower for the mid income level because they have more income to avoid defaults. The data on high and very high income default rates aren't useful because there isn't enough data.

#### How do different loan purposes affect on-time repayment of the loan?

In [15]:
pivot_table = df.groupby(['purpose_number']).agg({'debt':'mean'})
pivot_table

,debt
purpose_number,
0,0.093935
1,0.092484
2,0.074748
3,0.073259


In [22]:
related = ['car','education','wedding','real-estate']

print('The overall default rate is: {:.2f}%'.format(overall_rate))

for i in range(len(incomes)):
    print('The default rate for {:}-related loans is: {:.2f}%'.format(related[i], pivot_table['debt'][i]*100))

The overall default rate is: 8.12%
The default rate for car-related loans is: 9.39%
The default rate for education-related loans is: 9.25%
The default rate for wedding-related loans is: 7.47%
The default rate for real-estate-related loans is: 7.33%


The default rates for car and education related loans are slightly more likely than those for weddings and real-estate probably because real-estate and weddings are more luxury.

### Conclusion

It is somewhat possible to evaluate the possibility that someone would default on a loan to various degrees by their data on children, income level, and marital status. Those who are married or take loans for luxury purchases are less likely to default while those with lower income or children are more likely to default.